<h1 class = "alert alert-info">
    Adding position features to NBA original table<br/>
    <small>
        Sources: <br/>
        nba-2015-2016-regular-season.csv : <ah2 href =http://stats.nba.com/players/traditional/#!?sort=PTS&dir=-1&Season=2015-16&SeasonType=Regular%20Season>http://stats.nba.com/players/traditional/ </ah2><br/>
        Positions.csv : <a href = "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/year/2016" >http://www.espn.com/nba/statistics/player/</a><br/>
        SalaryESPN.csv : <a href = "http://www.espn.com/nba/salaries/_/year/2016" >http://www.espn.com/nba/salaries/</a>. <br/>
        Access on 02-02-2017
        <h6>(as data was incomplete, we assigned salaries from 2015 or 2017 for those players whose salary in 2016 was mmissing)<br/></h6>
        
    </small>
</h1>

In [39]:
import numpy as np
import pandas as pd

posic = pd.read_csv("Positions.csv", usecols=['Player','PosicAbbrev'])
sal = pd.read_csv("SalaryESPN.csv",usecols = ['NAME','SALARY'])
NBA =pd.read_csv("nba-2015-2016-regular-season.csv")

In [40]:
posic.head()

,Player,PosicAbbrev
0,Russell Westbrook,PG
1,Rajon Rondo,PG
2,Stephen Curry,PG
3,Michael Carter-Williams,PG
4,John Wall,PG


In [41]:
sal.head()

,NAME,SALARY
0,Kobe Bryant,25000000
1,LeBron James,22970500
2,Carmelo Anthony,22875000
3,Dwight Howard,22359364
4,Chris Bosh,22192730


In [42]:
NBA.head()

,Player,TEAM,AGE,GP,W,L,MIN,FGM,FGA,FG%,...,REB,AST,TOV,STL,BLK,PF,DD2,TD3,PTS,+/-
0,Aaron Brooks,CHI,31,69,36,33,16.1,2.7,6.8,40.1,...,1.5,2.6,1.2,0.4,0.1,1.9,0,0,7.1,-0.1
1,Aaron Gordon,ORL,20,78,32,46,23.9,3.5,7.4,47.3,...,6.5,1.6,0.8,0.8,0.7,2.0,12,0,9.2,-1.0
2,Aaron Harrison,CHA,21,21,15,6,4.4,0.2,0.9,26.3,...,0.7,0.1,0.2,0.3,0.0,0.5,0,0,0.9,-0.3
3,Adreian Payne,MIN,25,52,18,34,9.3,1.0,2.8,36.6,...,2.1,0.6,0.7,0.3,0.2,1.5,0,0,2.5,-2.3
4,Al Horford,ATL,30,82,48,34,32.1,6.5,12.8,50.5,...,7.3,3.2,1.3,0.8,1.5,2.0,17,0,15.2,2.8


## Joining NBA and posic
Solving conflicts

In [43]:
df = NBA.set_index('Player').join(posic.set_index('Player')).reset_index()
df.head()
conflits2 = df.loc[~df.loc[:,'PosicAbbrev'].isin((' PG',' SG',' PF',' C',' SF')),('Player')]
conflits2.sort_values(inplace=True)
conflits2 = conflits2.reset_index()['Player']

df = posic.set_index('Player').join(NBA.set_index('Player')).reset_index()
df.head()
conflits = df.loc[~(df.loc[:,'AGE'] >= 0),('Player')]
conflits.sort_values(inplace=True)
conflits = conflits.reset_index()['Player']

aka = pd.DataFrame()
aka['NBA'] = conflits
aka['Pos'] = conflits2

aux = aka.loc[6,'Pos']
aka.loc[6,'Pos'] = aka.loc[8,'Pos']
aka.loc[8,'Pos'] = aka.loc[7,'Pos']
aka.loc[7,'Pos'] = aux
df = NBA

for i in aka.index:
    df.loc[df['Player'] == aka.loc[i,'Pos'],'Player'] = aka.loc[i,'NBA']
#Finally joining nba data and position
df = df.set_index('Player').join(posic.set_index('Player')).reset_index()

Checking if all problems were solved

In [44]:
df.loc[:,'PosicAbbrev'].isin((' PG',' SG',' PF',' C',' SF')).all()

True

## Joining NBA and sal

In [45]:
# performing joint between nba data and salaries
df = df.set_index('Player').join(sal.set_index('NAME')).reset_index()

## To file

In [46]:
df.to_csv("../NBA-data-With-Positions.csv")